<a href="https://colab.research.google.com/github/jeevanshrestha/langchain/blob/main/Langchain_Chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Langchain Chains

### Initial Setup

#### Install Required Libraries


In [8]:
!pip install openai langchain langchain-community langchain_huggingface langchain-openai unstructured faiss-cpu -q
!pip install grandalf -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00


#### Mount Google Driev

In [2]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#set working directory
import os
os.chdir('/content/drive/MyDrive/GenAI/RAG/')

#### OpenAI Key Setup

In [4]:
from google.colab import userdata
api_key = userdata.get('genai_course')

#### Import Libraries


In [5]:
#Import the libraries
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from IPython.display import display, Markdown
import json
import numpy as np
import pandas as pd

In [6]:
print(os.getcwd())

/content/drive/MyDrive/GenAI/RAG


### Simple Chains

In [9]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template='Generate 5 interesting facts about {topic}',
    input_variables=['topic']
)

model = ChatOpenAI(api_key=api_key)

parser = StrOutputParser()

chain = prompt | model | parser

result = chain.invoke({'topic':'cricket'})

In [10]:


print(result)


1. Cricket is the second most popular sport in the world, with over 2.5 billion fans globally, second only to soccer.
2. The longest cricket match in history lasted for a grueling 14 days, played between England and South Africa in 1939.
3. The highest individual score in a Test match is held by Brian Lara, who scored 400 runs in a single innings against England in 2004.
4. The Ashes is one of the oldest and most prestigious cricket series, played between England and Australia since 1882.
5. The sport of cricket originated in England in the 16th century and has since spread to over 100 countries, becoming particularly popular in countries like India, Australia, and Pakistan.


In [11]:

chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  


### Sequential Chain

In [12]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt1 = PromptTemplate(
    template='Generate a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a 5 pointer summary from the following text \n {text}',
    input_variables=['text']
)

model = ChatOpenAI(api_key = api_key)

parser = StrOutputParser()

chain = prompt1 | model | parser | prompt2 | model | parser

result = chain.invoke({'topic': 'Unemployment in India'})


In [13]:

print(result)


1. The unemployment rate in India was 7.7% in April 2021, with youth unemployment at 22.5%.
2. Lack of job creation in the formal sector, skills gap, and Covid-19 pandemic are major causes of unemployment in India.
3. Potential solutions include encouraging entrepreneurship, investing in education, promoting job creation in key sectors, labor market reforms, and enhancing social protection programs.
4. India needs a multi-faceted approach involving government, private sector, and civil society to address unemployment effectively.
5. Collaboration and prioritization of job creation and skills development are crucial for reducing unemployment and providing better opportunities for the workforce in India.


In [14]:

chain.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
    +----------------+     
    | PromptTemplate |     
    +----------------+     
            *              
            *              
            *       

### Parallel Chains

In [16]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint

model1 = ChatOpenAI(api_key=api_key)

# Define the model
llm = HuggingFaceEndpoint(
    repo_id="google/gemma-2-2b-it",
    task="text-generation"
)

model2 = ChatHuggingFace(llm=llm)

prompt1 = PromptTemplate(
    template='Generate short and simple notes from the following text \n {text}',
    input_variables=['text']
)

prompt2 = PromptTemplate(
    template='Generate 5 short question answers from the following text \n {text}',
    input_variables=['text']
)

prompt3 = PromptTemplate(
    template='Merge the provided notes and quiz into a single document \n notes -> {notes} and quiz -> {quiz}',
    input_variables=['notes', 'quiz']
)

parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'notes': prompt1 | model1 | parser,
    'quiz': prompt2 | model2 | parser
})

merge_chain = prompt3 | model1 | parser

chain = parallel_chain | merge_chain

text = """
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

The advantages of support vector machines are:

Effective in high dimensional spaces.

Still effective in cases where number of dimensions is greater than the number of samples.

Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.

Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.

The disadvantages of support vector machines include:

If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.

SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).

The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({'text':text})




In [17]:
print(result)


Notes:

- Support vector machines (SVMs) are used for classification, regression, and outlier detection.
- Advantages of SVMs include effectiveness in high dimensional spaces, memory efficiency, and versatility with different Kernel functions.
- Disadvantages of SVMs include potential overfitting with too many features and the lack of direct probability estimates.
- It is recommended to use dense or sparse sample vectors as input for optimal SVM performance.

Quiz:

1. Q: What are the key tasks support vector machines (SVMs) excel at?
A: SVMs are adept at classification, regression, and outlier detection.

2. Q: How do SVMs address the challenge of high-dimensional data?
A: SVMs are effective in high-dimensional spaces because they leverage a subset of training points for the decision function (support vectors).

3. Q: What distinguishes SVM's memory efficiency?
A: Support vector machines are memory efficient because they use a subset of training points in their decision function.

4. 

In [18]:

chain.get_graph().print_ascii()

            +---------------------------+            
            | Parallel<notes,quiz>Input |            
            +---------------------------+            
                 **               **                 
              ***                   ***              
            **                         **            
+----------------+                +----------------+ 
| PromptTemplate |                | PromptTemplate | 
+----------------+                +----------------+ 
          *                               *          
          *                               *          
          *                               *          
  +------------+                 +-----------------+ 
  | ChatOpenAI |                 | ChatHuggingFace | 
  +------------+                 +-----------------+ 
          *                               *          
          *                               *          
          *                               *          
+-----------------+         

### Conditional Chain

In [19]:

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal


model = ChatOpenAI(api_key = api_key)

parser = StrOutputParser()

class Feedback(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='Give the sentiment of the feedback')

parser2 = PydanticOutputParser(pydantic_object=Feedback)

prompt1 = PromptTemplate(
    template='Classify the sentiment of the following feedback text into postive or negative \n {feedback} \n {format_instruction}',
    input_variables=['feedback'],
    partial_variables={'format_instruction':parser2.get_format_instructions()}
)

classifier_chain = prompt1 | model | parser2

prompt2 = PromptTemplate(
    template='Write an appropriate response to this positive feedback \n {feedback}',
    input_variables=['feedback']
)

prompt3 = PromptTemplate(
    template='Write an appropriate response to this negative feedback \n {feedback}',
    input_variables=['feedback']
)

branch_chain = RunnableBranch(
    (lambda x:x.sentiment == 'positive', prompt2 | model | parser),
    (lambda x:x.sentiment == 'negative', prompt3 | model | parser),
    RunnableLambda(lambda x: "could not find sentiment")
)

chain = classifier_chain | branch_chain


In [22]:
print(chain.invoke({'feedback': 'This is a very bad phone'}))


I'm sorry to hear that you had a negative experience. Please let us know how we can improve and make things right for you. Your feedback is valuable to us.


In [23]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
     +------------+      
     | ChatOpenAI |      
     +------------+      
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     


In [24]:
print(chain.invoke({'feedback': 'This is a fantastic phone'}))

Thank you so much for your kind words! We are thrilled to hear that you had a positive experience with us. We strive to provide excellent service and we are glad that we were able to meet your expectations. If you have any other feedback or suggestions for us, please feel free to share. Thank you for your support!


In [25]:
chain.get_graph().print_ascii()

    +-------------+      
    | PromptInput |      
    +-------------+      
            *            
            *            
            *            
   +----------------+    
   | PromptTemplate |    
   +----------------+    
            *            
            *            
            *            
     +------------+      
     | ChatOpenAI |      
     +------------+      
            *            
            *            
            *            
+----------------------+ 
| PydanticOutputParser | 
+----------------------+ 
            *            
            *            
            *            
       +--------+        
       | Branch |        
       +--------+        
            *            
            *            
            *            
    +--------------+     
    | BranchOutput |     
    +--------------+     
